In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import fsspec
import math
import matplotlib.pyplot as plt
from pathlib import Path

# display and warning messages
import xarray as xr
xr.set_options(display_style="html")  # display dataset nicely
import warnings
warnings.simplefilter("ignore")  # filter some warning messages

# Small style adjustments for more readable plots
plt.style.use("seaborn-whitegrid")
plt.rcParams["figure.figsize"] = (8, 6)
plt.rcParams["font.size"] = 14

# code features from https://scitools.org.uk/cartopy/docs/v0.14/matplotlib/feature_interface.html
crs = ccrs.PlateCarree()  # set projection

In [ ]:
# path to dataset
DATA_DIR = Path.home()/Path('shared/climate-data')
ds = xr.open_dataset("~/shared/climate-data/ds_hw.nc")

In [ ]:
# function for calculating offshore longitude
def offshore(lat, long, distance=1):
    radius_earth = 6371
    lat_rad = np.radians(lat)
    long_rad = np.radians(long)
    
    # calculate distance in longitude
    latitude_radius = radius_earth * math.cos(lat_rad)
    longitude = (long_rad - (distance / lat_rad)) * (180 / math.pi)
    
    return longitude

In [ ]:
# setting coordinates to Monterey, calculating offshore longitude 
latitude, longitude = 36.80, -121.94
offshore_longitude = offshore(latitude, longitude, distance=1)

In [ ]:
# celsius conversion
ds['sst_celsius'] = ds.analysed_sst - 273.15
ds.coords['day_of_year'] = ds.time.dt.dayofyear

In [ ]:
# making sure to select location using offshore longitude in this case
monterey = ds.sel(lat=latitude, lon=offshore_longitude, method='nearest')

In [ ]:
# rolling average on entire dataset
monterey_roll = monterey.sst_celsius.rolling(time = 31, center = True, min_periods = 1).mean()

In [ ]:
# filter by year
monterey_14 = monterey_roll.sel(time='2014')
monterey_15 = monterey_roll.sel(time='2015')
monterey_16 = monterey_roll.sel(time='2016')

# rolling mean over 2002-2013
period = monterey_roll.sel(time=slice('2002','2013')).groupby('day_of_year')
rolling_means = monterey_roll.sel(time=slice('2002','2013')).groupby('day_of_year').mean()

# maxes and mins
maxes = monterey_roll.sel(time=slice('2002','2013')).groupby('day_of_year').max()
mins = monterey_roll.sel(time=slice('2002','2013')).groupby('day_of_year').min()
rolling_means['maxes'] = maxes
rolling_means['mins'] = mins

# lower and upper bounds std
rollings_stds = monterey_roll.sel(time=slice('2002','2013')).groupby('day_of_year').std()
rollings_stds['lower'] = rolling_means - rollings_stds
rollings_stds['upper'] = rolling_means + rollings_stds

In [ ]:
# plot means for 2002-2013
rolling_means.plot(x='day_of_year', color='black', zorder=4)

# plot 2014, 2015, 2016
monterey_14.plot(x='day_of_year', color='blue')
monterey_15.plot(x='day_of_year', color='red')
monterey_16.plot(x='day_of_year', color='green')

# creating gray area of plot with min, man, stds
plt.fill_between(rollings_stds.day_of_year, rollings_stds.lower, rollings_stds.upper, zorder=3, color='gray')
plt.fill_between(rolling_means.day_of_year, rolling_means.mins, rolling_means.maxes, alpha=0.1, color='black')

# plot titles, labels, axes, etc
plt.title('36N ~Monterey')
plt.grid(False)
plt.xticks(np.arange(0, 480, 40), 
           ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D'])  
plt.xlabel('Month')
plt.ylim(6.5, 23.5)
plt.xlim(0, 370)
plt.ylabel('SST_offshore(°C)')
plt.legend(loc='upper left', labelcolor='linecolor', handlelength=0)

plt.savefig('figures/q06_plot.png')